# Get the target words in comma seperated list

In [43]:
print ("######################")
print ("Make sure you have internet connection (2-5 Mbps),Tensorflow, math, numpy, NLTK (with data),random, os,six.moves libraries in Python 3.6.3 exists.")
print ("######################")
target_list = input("Enter comma seperated words for which you want to predict the nearest words: ")

######################
Make sure you have internet connection (2-5 Mbps),Tensorflow, math, numpy, NLTK (with data),random, os,six.moves libraries in Python 3.6.3 exists.
######################
Enter comma seperated words for which you want to predict the nearest words: small


In [52]:
print ("######################")
download_embed = input("To download pre-trained embeddings(For faster predictions), enter 1, else 0 for training 100000 iterations over data ::")
print ("######################")

######################
To download pre-trained embeddings(For faster predictions), enter 1, else 0 for training 100000 iterations over data ::1
######################


In [44]:
print ("Tokenizing inputs in to words")
from nltk.tokenize import word_tokenize
target_tokens = word_tokenize(target_list)
print(target_tokens)

Tokenizing inputs in to words
['small']


In [45]:
print ("Convert to lower case")
target_tokens = [w.lower() for w in target_tokens]
print(target_tokens)

Convert to lower case
['small']


In [46]:
# remove punctuation from each word
import string
table = str.maketrans('', '', string.punctuation)
target_stripped = [w.translate(table) for w in target_tokens]
print ("remove punctuation from each word, if any",target_stripped)

remove punctuation from each word, if any ['small']


In [47]:
# remove remaining tokens that are not alphabetic
target_words = [word for word in target_stripped if word.isalpha()]
print ("remove remaining tokens that are not alphabetic, if any",target_words)

remove remaining tokens that are not alphabetic, if any ['small']


In [48]:
#Converting the target_words to tuple as that is what the further functions expect.
target_words=tuple(target_words)

# Libraries

In [8]:
#Run these. 
#Still there are many more libraries which I have placed in code in appropriate sections for ease of readability.
import os
import random
from six.moves import range
import numpy as np
import collections

# Download the Corpus to train.

    The Data placed in your Google Drive and shared. You can get the id from URL itself.

In [9]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination) 

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)



In [10]:
#Download the file if not yet downloaded.
   #https://drive.google.com/file/d/1rI79RgJx2ZajOGtYsw-kxCjrr2VUYH1e/view?usp=sharing
   #id=1rI79RgJx2ZajOGtYsw-kxCjrr2VUYH1e
import os.path
if((os.path.isfile("corpus.txt")!=True)):
   id="1rI79RgJx2ZajOGtYsw-kxCjrr2VUYH1e"
   destination="corpus.txt"
   download_file_from_google_drive(id,destination)

In [41]:
#Download embeddings.
#https://drive.google.com/file/d/1LiBLVn1Q3-a2GndvnA8-vRlTR9kNBOdE/view?usp=sharing
import os.path
if(int(download_embed)==1):
    id="1LiBLVn1Q3-a2GndvnA8-vRlTR9kNBOdE"
    destination="skip_embed.py"
    download_file_from_google_drive(id,destination)
elif ((os.path.isfile("skip_embed.py")==True) & (int(download_embed)!=1) ):
    os.remove("skip_embed.py")
else:
    print ("Unknown error")

## Corpus Reading and Cleaning Process.

In [11]:
print ("######################")
if ((os.path.isfile("dictionary.npy")==True) & (os.path.isfile("reverse_dictionary.npy")==True)):
    print("The corpus will not be processed as it has been already processed")
else:
    print("Reading and cleaning corpus to train")
    file = open('corpus.txt', 'r')
    text = file.read()
    file.close()
    #NLTK for text processing
    import nltk
    print("Removing non english words.")
    eng_words = set(nltk.corpus.words.words())

    text_english=" ".join(w for w in nltk.wordpunct_tokenize(text) \
             if w.lower() in eng_words or not w.isalpha())
    #Text Tokenization
    print("Split into words")
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(text_english)
    print("Convert to lower case")
    tokens = [w.lower() for w in tokens]
    print("Remove punctuation from each word")
    import string
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    print("Remove remaining tokens that are not alphabetic")
    words = [word for word in stripped if word.isalpha()]
    print("Filter out stop words")
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    print("Removed Single Letter words")
    single_letters=('b','c','d','e','f','g','h','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z')
    words=[w for w in words if not w in single_letters]
    print("Total words in corpus")
    print (len(words))
    print("Total unique words")
    print (len(set(words)))
    #Word Dictionary and Anlysis of the words.
    vocabulary_size = len(set(words))
    import collections

    def build_dataset(words):
      count = [['UNK', -1]]
      count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
      dictionary = dict()
      for word, _ in count:
        dictionary[word] = len(dictionary)
      data = list()
      unk_count = 0
      for word in words:
        if word in dictionary:
          index = dictionary[word]
        else:
          index = 0  # dictionary['UNK']
          unk_count = unk_count + 1
        data.append(index)
      count[0][1] = unk_count
      reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
      return data, count, dictionary, reverse_dictionary

    data, count, dictionary, reverse_dictionary = build_dataset(words)
    print('Most common words (+UNK)', count[:5])
    print('Sample data', data[:10])
    import numpy as np
    # Save
    np.save('dictionary.npy', dictionary)
    np.save('reverse_dictionary.npy', reverse_dictionary)
    np.save('data.npy', data)

######################
The corpus will not be processed as it has been already processed


In [12]:
print ("######################")
print ("Loading Data, Dictionary & Reverse Dictionary:")
data=(list((np.load("data.npy"))))
dictionary = np.load('dictionary.npy').item()
reverse_dictionary = np.load('reverse_dictionary.npy').item()

######################
Loading Data, Dictionary & Reverse Dictionary:


# Create Indexed Dictionary, Reverse Dictionary, Analysis of Frequently Used Word Counts.

# Batch Generation
    
    Batches of data will be fed in each iteration of tensorflow. Every iteration will get new batch of data, which is controled by global data_index variable, which gets updated in each run of generate_batch function.

In [25]:
import numpy as np

#This is a important variable. This gets updated everytime batch function is run at global context.
#Basically data_index after one run, from where of data I need to select the next batch.
#So in tf for each iteration when you call batch you will get unique batch till the end of the data.
data_index = 0
#Creating multiple batches of rows=batch_size for batch learning.
#We will be calling this and passing batch size while calling TF session.

#Please note: This function is getting "data" from earlier defined and populated data variable.

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the Center.
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

#The sample of data.
print('data:', [reverse_dictionary[di] for di in data[:4]])

print ("\n Showing how the skip windows and skips transform training data: \n")
for num_skips, skip_window in [(1, 1)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=4, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    features:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(4)])

data: ['selected', 'eyewear', 'noise', 'insertion']

 Showing how the skip windows and skips transform training data: 


with num_skips = 1 and skip_window = 1:
    features: ['eyewear', 'noise', 'insertion', 'loss']
    labels: ['selected', 'eyewear', 'loss', 'insertion']


In [27]:
batch.shape

(4,)

In [36]:
#Defining batch_size, embedding size(word vector size), skip window, negative sample size.
batch_size = 150
embedding_size = 150 # Dimension of the embedding vector.
skip_window = 1 # words to consider left and right.
num_skips = 1 # How many times to reuse an input to generate a label
num_sampled = 80 # Number of negative examples to sample.
vocabulary_size=len(dictionary)

# Define Tensorflow Computational Graph

In [37]:
import tensorflow as tf
import math
graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):

        # Input data.
        train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

        # Variables.
        # embedding, vector for each word in the vocabulary
        #Remember that you have defined vocabulary_size = 7 when you created function build dataset
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        
        softmax_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
        
        softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))

        
        # Looking up for embeddings for train dataset
        embed = tf.nn.embedding_lookup(embeddings, train_dataset)

        # Compute the softmax loss, using a sample of the negative labels each time.
        # inputs are embeddings of the train words
        # with this loss we optimize weights, biases, embeddings
        
        
        #loss = tf.reduce_mean(tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=avg_embed,
         #                      labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))
            #After doing the literature study, found out nce_loss is better than sampled_softmax_loss.
        # Compute the softmax loss, using a sample of the negative labels each time.
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=softmax_weights,
                 biases=softmax_biases,
                 labels=train_labels,
                 inputs=embed,
                 num_sampled=num_sampled,
                 num_classes=vocabulary_size))

        # Optimizer.
        optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)

        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm

# Train using Tensorflow Session on Batches of Data

In [40]:
#Run
#Iterations
import os.path
num_steps = 100000
if ((os.path.isfile("skip_embed.npy")!=True)):
    print ("########################")
    print("We will run Tensor Flow session and create & save embeddings in embed.py, as it doesn't exist.\n Will take more than 10-15 minutes for 100000 iterations, depending upon your system config. Please wait for predictions. \n")
    print ("########################")
    with tf.Session(graph=graph) as session:
            tf.global_variables_initializer().run()
            print('Session Initialized')
            average_loss = 0
            for step in range(num_steps):
                batch_data, batch_labels = generate_batch(batch_size, num_skips=num_skips, skip_window=skip_window)
                feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
                #I am only interested to get loss in return in variable 1 and add it to average loss.
                _, l = session.run([optimizer, loss], feed_dict=feed_dict)
                average_loss += l
                if step % 2000 == 0:
                    if step > 0:
                        average_loss = average_loss / 2000
                        # The average loss is an estimate of the loss over the last 2000 batches.
                    print('Average loss at step %d: %f' % (step, average_loss))
                    average_loss = 0
                # note that this is expensive (~20% slowdown if computed every 500 stepss
                final_embeddings = normalized_embeddings.eval()
                np.save("skip_embed.npy", final_embeddings)
else:
    print ("Already embedding file trained earlier/downloaded exists. Skipping training process.")

Already embedding file trained earlier/downloaded exists. Skipping training process.


# Prediction Function

In [41]:
#Creating predict function
#Input: list of words, final_embeddings,reverse_dictionary,top_nearest_words_needed
#Output: The nearest 10 words in sorted order.
def predicted_words(target_words,final_embeddings,dictionary,reverse_dictionary,top_nearest_words_needed):

    #Handling the single input and multiple inputs, both.
    target_list=[]
    if (type(target_words)==tuple):
        for i in range(0,len(target_words)):
            target_list.append(target_words[i])
    else:
        target_list.append(target_words)
    
    #Take word by word and predict list of nearest words.
    for i in range(0,len(target_list)):
        #Cleaning target word.
        target=target_list[i]
        #search for index in dictionary. If not found assign it to unknown word.
        #The results will be obviously not accurate, but will not give annoying not found error.
        #Approach is debatable! 
        #But I am following approach in google keypad, when I enter non existing word, it will still give some junk predictions.
        if(dictionary.get(target)==None):
            target_embedding=final_embeddings[0,:]
        else:
            target_embedding=final_embeddings[int(dictionary.get(target)),:]
        
        if(dictionary.get(target)!=None):
            #Avoiding Comparing Similarity with Unknown Vector
            #cosine_similarity
            cosine_similarity=np.matmul(target_embedding,np.transpose(final_embeddings[1:final_embeddings.shape[0],:]))
        
            #I am building a array with index in one column and cosine similarity in another column.
            word_index=np.zeros((cosine_similarity.shape[0],1), dtype=int)
        
            for j in range(0,cosine_similarity.shape[0]):
                word_index[j]=j+1
        
            #Reshaping cosine_similarity to match with word index array.
            cosine_similarity_reshape=cosine_similarity.reshape(cosine_similarity.shape[0],1)
        
            #Appending & Sorting.
            target_sim_array=np.append(word_index,cosine_similarity_reshape,axis=1)
            target_sim_array_sorted=target_sim_array[target_sim_array[:, 1].argsort()]
        
            #Taking top nearest word index except word itself (0 th position)
            top_word_index=target_sim_array_sorted[:,0][-(top_nearest_words_needed+1):-1:]
        
            #Top word predictions using reverse array and reverse dictionary
            predicted_words=list()
            for k in reversed(top_word_index):
                predicted_words.append(reverse_dictionary[k])
            
            print ("\n Nearest words for word: %s" %target, ": ordered by nearest word predicted first is %s" %predicted_words)
        else:
            
            #cosine_similarity
            cosine_similarity=np.matmul(final_embeddings[0,:],np.transpose(final_embeddings))
        
            #I am building a array with index in one column and cosine similarity in another column.
            word_index=np.zeros((cosine_similarity.shape[0],1), dtype=int)
        
            for j in range(0,cosine_similarity.shape[0]):
                word_index[j]=j
        
            #Reshaping cosine_similarity to match with word index array.
            cosine_similarity_reshape=cosine_similarity.reshape(cosine_similarity.shape[0],1)
        
            #Appending & Sorting.
            target_sim_array=np.append(word_index,cosine_similarity_reshape,axis=1)
            target_sim_array_sorted=target_sim_array[target_sim_array[:, 1].argsort()]
        
            #Taking top nearest word index except word itself (0 th position)
            top_word_index=target_sim_array_sorted[:,0][-(top_nearest_words_needed+1):-1:]
        
            #Top word predictions using reverse array and reverse dictionary
            predicted_words=list()
            for k in reversed(top_word_index):
                predicted_words.append(reverse_dictionary[k])
            
            print ("\n The word is unknown as of now. Still, based on cosine distance for unknown word vector, Nearest for words for unknown word: %s" %target, ": ordered by nearest word predicted first is %s" %predicted_words)  
            
        

# Test

In [42]:
target_words

('small',)

In [49]:
import numpy as np
top_nearest_words_needed=10
# Load embeddings, dictionary and reverse dictionary saved earlier in files.
# Advantage is you can reduce run time, if these files doesn't exist.
final_embeddings=np.load("skip_embed.npy")
dictionary = np.load('dictionary.npy').item()
reverse_dictionary = np.load('reverse_dictionary.npy').item()

In [50]:
print ("###########################")
predicted_words(target_words,final_embeddings,dictionary,reverse_dictionary,top_nearest_words_needed)

###########################

 Nearest words for word: small : ordered by nearest word predicted first is ['large', 'asperity', 'although', 'abalone', 'isochrone', 'electroshock', 'smaller', 'unbound', 'leat', 'jinja']
